In [ ]:
... imports ...

from photutils.psf import build_psf

In [ ]:
# do whatever data loading is required. 
...
list_of_ndd_obj = load_data(...)  
# The end result should be a list of NDData objects containing all the 
# dithers in one filter for that observation

# Baseline object

To follow the rest of the PSF machinery, the structure here is to create a class where all the parameters get set, and then run that class on the data.

## Creating the builder 

In [ ]:
psf_builder = PSFBuilder(model=Gaussian2D())
# Or for something like EPSF where they fitting might need to be special, you might do
psf_builder = PSFBuilder(model=EPSF(), fitter=EPPSFFitter(...))

In [ ]:
# it's here that you'd specify any additional parameters/specify extra algorithm details, etc.
psf_builder = PSFBuilder(model=Gaussian2D(), psfstar_threshold=-10*u.mag)

# as a stretch goal, it would be neat if this were smart enough to use units to make input handling easier:
psf_builder = PSFBuilder(model=Gaussian2D(), psfstar_threshold=1e4*u.adu)

You could also use this to specify specific algorithms for sub-stanges of the process.  For example, if I wanted a specific star finder: e.g.,

In [ ]:
psf_builder = PSFBuilder(model=EPSF(), fitter=EPPSFFitter(...), star_finder=IRAFStarFinder(...))

## Running the builder 

In [ ]:
psf_model, psf_stars = psf_builder(list_of_ndd_obj)

`psf_stars` is an astropy table that looks like what the star-finder outputs, with aperture photometry and the results of the final fitting included. It would have a final column called something like `'is_psf_star'`, and if True, 

The user might look at `psf_stars` and decide some of the stars are good/bad, in which case they can fiddle around with the `'is_psf_star'` column and do the below to re-fit (skips the star-finding step)

In [ ]:
psf_model, psf_stars2 = psf_builder(list_of_ndd_obj, psf_stars=psf_stars)

In this case, no star-finder would get run, it would just use what's there.  Users

The key end result is the `psf_model`, which is a 2D Astropy model that maps x,y to the PSF flux.  This is the same  current `photutils.psf` models like `FittableImageModel`. Should be normalized so that if the `flux` parameter is 1, the PSF's total flux is 1. Note that this *might* not mean the ePSF sums to 1: this allows aperture corrections to be folded into the model.



# Heuristic high-level function 

A `build_psf` function here would have to make a lot of assumptions about the data, etc.  So the function would probably need to contain a lot of heuristic bits to guess detection thresholds,  etc.

In [ ]:
psf_model = build_psf(list_of_ndd_obj)